# Extended RNA-Seq Analysis Training Demo

## Overview

For simplicity and time, The short tutorial workflow uses truncated and partial run data from the Cushman et al., project.

The tutorial repeats the short tutorial, but with the full fastq files and includes some extra steps, such as how to download and prepare the transcriptome files used by salmon, alternate ways to navigate the NCBI databases for annotation or reference files you might need, and how to combine salmon outputs at the end into a single genecount file.

Full fastq files can be rather large, and so the downloading, extracting, and analysis of them means this tutorial can take over 1 hour 45 minutes to run the code fully. This is part of the reason we have a short and easy introductory tutorial, and this longer more full tutorial for those interested.

If this is too lengthy feel free to move on to the snakemake tutorial or the DEG analysis tutorial -- all the files used in the DEG tutorial were created using this extended tutorial workflow.

![RNA-Seq workflow](images/rnaseq-workflow.png)

### STEP 1: Install Mambaforge

First install Mambaforge.


In [1]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge
!date +"%T"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 82.9M  100 82.9M    0     0   116M      0 --:--:-- --:--:-- --:--:--  256M
PREFIX=/home/jupyter/mambaforge
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2023.7.22-hbcca054_0.conda
Extracting ld_impl_linux-64-2.40-h41732ed_0.conda
Extracting libstdcxx-ng-13.1.0-hfd8a6a1_0.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.10-3_cp310.conda
Extracting tzdata-2023c-h71feb2d_0.conda
Extracting libgomp-13.1.0-he5830b7_0.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-ng-13.1.0-he5830b7_0.conda
Extracting bzip2-1.0.8-h7f98852_4.tar.bz2
Extracting c-ares-1.19.1-hd590300_0.conda
Extracti

Next, using mambaforge and bioconda, install the tools that will be used in this tutorial.

In [2]:
#tell the computer where the mambaforge bin files are located
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"

#now we can easily use 'mamba' command to install software 
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc salmon entrez-direct gffread parallel-fastq-dump sra-tools


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.2) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['trimmomatic', 'fastqc', 'multiqc', 'salmon', 'entrez-direct', 'gffread', 'parallel-fastq-dump', 's

### STEP 2: Setup Environment

Create a set of directories to store the reads, reference sequence files, and output files. Notice that first we remove the `data` directory to clean up files from Tutorial_1


In [3]:
! cd $HOMEDIR
! echo $PWD
! rm -r data/
! mkdir -p data
! mkdir -p data/raw_fastq
! mkdir -p data/trimmed
! mkdir -p data/fastqc
! mkdir -p data/aligned
! mkdir -p data/reference
! mkdir -p data/fastqc_samples
! mkdir -p data/multiqc_samples

/home/jupyter


Set # THREADS depending on your VM size

In [19]:
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'

#python variable to hold the amount of threads your cpu has,
#useful for downstream tools like salmon, trimmomatic, etc
threads = int(numthreads[0])

#its also good to have a shell version of the variable for commands that use piping, 
#in jupyter, shell commandds with piping sometimes causes python variables to not work and generally be wonky.
%env THREADS=$threads

env: THREADS=3


### STEP 3: Downloading relevant FASTQ files using SRA Tools

Next we will need to download the relevant fastq files.

Because these files can be large, the process of downloading and extracting fastq files can be quite lengthy.

The sequence data for this tutorial comes from work by Cushman et al., <em><a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8191103/'>Increased whiB7 expression and antibiotic resistance in Mycobacterium chelonae carrying two prophages</a><em>.

We will be downloading the sample runs from this project using SRA tools, downloading from the NCBI's SRA (Sequence Run Archives).

However, first we need to find the associated accession numbers in order to download.


### STEP 3.1: Finding run accession numbers.

The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). To download runs, we will need the accession ID for each run we wish to download. 

The Cushman et al., project contains 12 runs. To make it easier, these are the run IDs associated with this project:

+ SRR13349122
+ SRR13349123
+ SRR13349124
+ SRR13349125
+ SRR13349126
+ SRR13349127
+ SRR13349128
+ SRR13349129
+ SRR13349130
+ SRR13349131
+ SRR13349132
+ SRR13349133


In this case, all these runs belong to the SRP (Sequence Run Project): SRP300216.

Sequence run experiments can be searched for using the SRA database on the NCBI website; and article-specific sample run information can be found in the supplementary section of that article.

For instance, here, the the authors posted a link to the sequence data GSE (Gene Series number), <a href='https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE164210'>GSE164210</a>. This leads to the appropriate 'Gene Expression Omnibus' page where, among other useful files and information, the relevant SRA database link can be found. 

Once the accession numbers are located, one can make a text file containing the list of accession IDs however they like.

Once again, to make things easier, we have made a .txt with these IDs that you can simply download here:

In [20]:
!gsutil cp gs://rnaseq-myco-bucket/reference/accs.txt .
!cat accs.txt

Copying gs://rnaseq-myco-bucket/reference/accs.txt...
/ [1 files][  144.0 B/  144.0 B]                                                
Operation completed over 1 objects/144.0 B.                                      
SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133


You can can also use BigQuery to generate an accession list following the instructions outlined in [this notebook](https://github.com/STRIDES/NIHCloudLabGCP/blob/main/tutorials/notebooks/SRADownload/SRA-Download.ipynb).

### STEP 3.1.2 (Optional): Generate the accession list file with BigQuery
This step uses Python. We will use the BigQuery API. 

We will create a client the using default project. 

Then we will query BigQuery using the species name and a range of accession numbers associated with this particular study. 

Feel free to play around with the query to generate different variations of accession numbers!

Please note that if you have errors to make sure you have this API enabled. You can search for BigQuery by navigating back to the Google Cloud Platform dashboard, back to the Google Cloud Platform, and using the search bar at the top, search for 'BigQuery'. On the BigQuery page, click  `Enable`

In [7]:
# Import the biquery api
from google.cloud import bigquery
import pandas

# Designate the client for the API
client = bigquery.Client(location="US")
print("Client created using default project: {}".format(client.project))

Client created using default project: kinglab-maine-strides


In [11]:
#Construct the query
query = """
#standardSQL
SELECT *
FROM `nih-sra-datastore.sra.metadata`
WHERE organism = 'Mycobacteroides chelonae'
and acc LIKE '%SRR133491%'
"""

#Run the query
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

#Put results of query into a dataframe
df = query_job.to_dataframe()
df  = df.sort_values(by='acc')

#print dataframe
df

,acc,assay_type,center_name,consent,experiment,sample_name,instrument,librarylayout,libraryselection,librarysource,...,geo_loc_name_sam,ena_first_public_run,ena_last_update_run,sample_name_sam,datastore_filetype,datastore_provider,datastore_region,attributes,run_file_version,jattr
2,SRR13349122,RNA-Seq,GEO,public,SRX9775146,GSM5004088,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[sra, fastq, run.zq]","[s3, gs, ncbi]","[gs.US, ncbi.public, s3.us-east-1]","[{'k': 'geo_accession_exp', 'v': 'GSM5004088'}...",1,"{""geo_accession_exp"": [""GSM5004088""], ""bases"":..."
7,SRR13349123,RNA-Seq,GEO,public,SRX9775146,GSM5004088,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[fastq, run.zq, sra]","[s3, gs, ncbi]","[gs.US, s3.us-east-1, ncbi.public]","[{'k': 'geo_accession_exp', 'v': 'GSM5004088'}...",1,"{""geo_accession_exp"": [""GSM5004088""], ""bases"":..."
8,SRR13349124,RNA-Seq,GEO,public,SRX9775147,GSM5004089,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[run.zq, fastq, sra]","[s3, gs, ncbi]","[s3.us-east-1, gs.US, ncbi.public]","[{'k': 'geo_accession_exp', 'v': 'GSM5004089'}...",1,"{""geo_accession_exp"": [""GSM5004089""], ""bases"":..."
4,SRR13349125,RNA-Seq,GEO,public,SRX9775147,GSM5004089,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[fastq, run.zq, sra]","[ncbi, gs, s3]","[s3.us-east-1, ncbi.public, gs.US]","[{'k': 'geo_accession_exp', 'v': 'GSM5004089'}...",1,"{""geo_accession_exp"": [""GSM5004089""], ""bases"":..."
3,SRR13349126,RNA-Seq,GEO,public,SRX9775148,GSM5004090,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[fastq, sra, run.zq]","[ncbi, s3, gs]","[s3.us-east-1, ncbi.public, gs.US]","[{'k': 'geo_accession_exp', 'v': 'GSM5004090'}...",1,"{""geo_accession_exp"": [""GSM5004090""], ""bases"":..."
9,SRR13349127,RNA-Seq,GEO,public,SRX9775148,GSM5004090,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[run.zq, fastq, sra]","[s3, ncbi, gs]","[gs.US, ncbi.public, s3.us-east-1]","[{'k': 'geo_accession_exp', 'v': 'GSM5004090'}...",1,"{""geo_accession_exp"": [""GSM5004090""], ""bases"":..."
6,SRR13349128,RNA-Seq,GEO,public,SRX9775149,GSM5004091,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[sra, fastq, run.zq]","[s3, gs, ncbi]","[gs.US, s3.us-east-1, ncbi.public]","[{'k': 'geo_accession_exp', 'v': 'GSM5004091'}...",1,"{""geo_accession_exp"": [""GSM5004091""], ""bases"":..."
1,SRR13349129,RNA-Seq,GEO,public,SRX9775149,GSM5004091,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[fastq, run.zq, sra]","[ncbi, gs, s3]","[ncbi.public, s3.us-east-1, gs.US]","[{'k': 'geo_accession_exp', 'v': 'GSM5004091'}...",1,"{""geo_accession_exp"": [""GSM5004091""], ""bases"":..."
11,SRR13349130,RNA-Seq,GEO,public,SRX9775150,GSM5004092,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[fastq, sra, run.zq]","[gs, s3, ncbi]","[gs.US, ncbi.public, s3.us-east-1]","[{'k': 'geo_accession_exp', 'v': 'GSM5004092'}...",1,"{""geo_accession_exp"": [""GSM5004092""], ""bases"":..."
5,SRR13349131,RNA-Seq,GEO,public,SRX9775150,GSM5004092,Illumina HiSeq 2500,PAIRED,cDNA,TRANSCRIPTOMIC,...,[],[],[],[],"[fastq, sra, run.zq]","[ncbi, gs, s3]","[gs.US, s3.us-east-1, ncbi.public]","[{'k': 'geo_accession_exp', 'v': 'GSM5004092'}...",1,"{""geo_accession_exp"": [""GSM5004092""], ""bases"":..."


We can then use some more python to write this dataframe to a text file, for easy use with our shell commands later.

In [12]:
#write the SRR column to a text file
with open('accs.txt', 'w') as f:
    accs = df['acc'].to_string(header=False, index=False)
    f.write(accs)
    
#print the text file
!cat accs.txt

SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133

### STEP 3.2: Using the SRA-toolkit for a single sample.

Sequence run accession IDs can be used to download sequence data, using the 'prefetch' tool of the SRA-toolkit.


In [13]:
#the 'prefetch' command downloads an SRA file.
!prefetch SRR13349123 -O data/raw_fastq -f yes


2023-10-06T04:46:08 prefetch.3.0.8: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2023-10-06T04:46:08 prefetch.3.0.8: 1) Downloading 'SRR13349123'...
2023-10-06T04:46:08 prefetch.3.0.8: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2023-10-06T04:46:08 prefetch.3.0.8:  Downloading via HTTPS...
2023-10-06T04:46:35 prefetch.3.0.8:  HTTPS download succeed
2023-10-06T04:46:36 prefetch.3.0.8:  'SRR13349123' is valid
2023-10-06T04:46:36 prefetch.3.0.8: 1) 'SRR13349123' was downloaded successfully
2023-10-06T04:46:36 prefetch.3.0.8: 'SRR13349123' has 0 dependencies


Here we can see the command for downloading a single SRA file using an acecssion ID 'SRR13349123'

Notice the SRA archives sequence files in the SRA format. 
Typically genome workflows process data in the form of zipped or unzipped .fastq, or .fasta files
So before we move on, we need to convert the files from .sra to .fastq.

There are multiple ways to do this. 

Included in the sra toolskit are fastq-dump and fasterq-dump. These convert SRA to FASTQ.

If you use fasterq-dump, its recommended to zip your fastq files after they are created.

There is also a tool called 'parallel-fastq-dump' which supports zipping the fastq files automatically into fastq.gz files, 

In [37]:
#convert sra to fastq
!fasterq-dump data/raw_fastq/SRR13349123 -f -O data/raw_fastq/
#compress fastq to fastq.gz to save space
!gzip data/raw_fastq/SRR13349123_1.fastq
!gzip data/raw_fastq/SRR13349123_2.fastq

spots read      : 11,165,256
reads read      : 22,330,512
reads written   : 22,330,512


### STEP 3.3 Downloading multiple files using the SRA-toolkit.

Often one wants to, as in our case, wish to download multiple runs at once.

To aid in this, SRA-tools supports batch downloading. This is why we created the text file earlier.

We can download multiple SRA files using a single line of code by using our list SRA IDs, and inputting that into the prefetch command.

And then feed that list into the sra-toolkit prefetch command. Note, it may take some time to download all the fastq files.

In [21]:
!prefetch --option-file accs.txt -O data/raw_fastq -f yes


2023-10-06T21:40:19 prefetch.3.0.8: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2023-10-06T21:40:19 prefetch.3.0.8: 1) Downloading 'SRR13349122'...
2023-10-06T21:40:19 prefetch.3.0.8: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2023-10-06T21:40:19 prefetch.3.0.8:  Downloading via HTTPS...
2023-10-06T21:40:43 prefetch.3.0.8:  HTTPS download succeed
2023-10-06T21:40:45 prefetch.3.0.8:  'SRR13349122' is valid
2023-10-06T21:40:45 prefetch.3.0.8: 1) 'SRR13349122' was downloaded successfully
2023-10-06T21:40:45 prefetch.3.0.8: 'SRR13349122' has 0 dependencies

2023-10-06T21:40:45 prefetch.3.0.8: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2023-10-06T21:40:45 prefetch.3.0.8: 2) Downloading 'SRR13349123'...
2023-10-06T21:40:45 prefetch.3.0.8: SRA Normalized Format file is being retrieved, if this is 

### STEP 3.4 Converting Multiple SRA files to Fastq

Fasterq-dump does not natively support batch converting of files.

There are several ways we can get around this.

For instance, one could use loops, or utilize piping.

The below code uses a 'for' loop to iterate through all the accession IDs in our accs.txt file.

It also adds includes various flags

-e for cpu threads
-m for maximum memory useage
-f to force overwrite
-O output directory. 

This process should take about 35 minutes or so.

In [22]:
!for x in `cat accs.txt`; do fasterq-dump -f -O data/raw_fastq -e $THREADS -m 4G data/raw_fastq/$x/$x.sra; done

##example of how to alternatively do the above process with parallel-fastq-dump using piping
#!cat accs.txt | xargs -I {} parallel-fastq-dump -O data/raw_fastq/ --tmpdir . --threads $THREADS --gzip --split-files --sra-id {}

spots read      : 10,827,590
reads read      : 21,655,180
reads written   : 21,655,180
spots read      : 11,165,256
reads read      : 22,330,512
reads written   : 22,330,512
spots read      : 10,727,273
reads read      : 21,454,546
reads written   : 21,454,546
spots read      : 10,992,686
reads read      : 21,985,372
reads written   : 21,985,372
spots read      : 12,267,497
reads read      : 24,534,994
reads written   : 24,534,994
spots read      : 12,563,032
reads read      : 25,126,064
reads written   : 25,126,064
spots read      : 12,652,387
reads read      : 25,304,774
reads written   : 25,304,774
spots read      : 12,961,793
reads read      : 25,923,586
reads written   : 25,923,586
spots read      : 10,083,015
reads read      : 20,166,030
reads written   : 20,166,030
spots read      : 10,491,160
reads read      : 20,982,320
reads written   : 20,982,320
spots read      : 11,341,357
reads read      : 22,682,714
reads written   : 22,682,714
spots read      : 11,603,881
reads read    

As before, it is good practice to turn .fastq files into .fastq.gz files to save space.

In our case, we will actually need to concatenate the fastq files later on, and so will zip after this.

The no redundant SRA files can also be deleted to save more space.

In [23]:
#find and delete all SRR subfolders in the raw_fastq directory
!find data/raw_fastq -type d -name 'SRR*' -exec rm -rf {} \;

find: ‘data/raw_fastq/SRR13349130’: No such file or directory
find: ‘data/raw_fastq/SRR13349127’: No such file or directory
find: ‘data/raw_fastq/SRR13349129’: No such file or directory
find: ‘data/raw_fastq/SRR13349123’: No such file or directory
find: ‘data/raw_fastq/SRR13349128’: No such file or directory
find: ‘data/raw_fastq/SRR13349131’: No such file or directory
find: ‘data/raw_fastq/SRR13349132’: No such file or directory
find: ‘data/raw_fastq/SRR13349125’: No such file or directory
find: ‘data/raw_fastq/SRR13349133’: No such file or directory
find: ‘data/raw_fastq/SRR13349126’: No such file or directory
find: ‘data/raw_fastq/SRR13349124’: No such file or directory
find: ‘data/raw_fastq/SRR13349122’: No such file or directory


### STEP 4: Copy reference transcriptome files that will be used by Salmon using E-Direct

Salmon is a tool that aligns RNA-Seq reads to a transcriptome.

So we will need a transcriptome reference file.

To get one, we can search through the NCBI assembly database, find an assembly, and download transcriptome reference files from that assembly using FTP links.

For instance, we will use the <a href='https://www.ncbi.nlm.nih.gov/assembly/GCF_001632805.1'>ASM163280v1</a> refseq assembly, found by searching through the NCBI assembly database. The FTP links can be accessed through the website in various ways, one way is to click the 'FTP directory for RefSeq assembly' link, found under 'Access the data', section.

Alternatively, if one were inclined, one could take the less common route and perform this through the NCBI command line tool suite called 'Entrez Direct' (EDirect).

This is an intricate and complicated set of tools, with many ways to do any one thing.

Below is an example of using an eDirect search query with a refseq identifier to obtain the relevant FTP directory, and then using that to download desired reference files.

In [43]:
#parse for the ftp link and download the genome reference fasta file

!esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.fna.gz " $0"/"$NF"_genomic.fna.gz"}' \
| bash

#parse for the ftp link and download the gtf reference fasta file

!esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.gff.gz " $0"/"$NF"_genomic.gff.gz"}' \
| bash

# parse for the ftp link and download the feature-table reference file 
# (for later use for merging readcounts with gene names in R code).

!esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_feature_table.txt.gz " $0"/"$NF"_feature_table.txt.gz"}' \
| bash


#unzip the compresseed fasta files

!gzip -d data/reference/*.gz --force

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1436k  100 1436k    0     0  1718k      0 --:--:-- --:--:-- --:--:-- 1716k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  376k  100  376k    0     0   440k      0 --:--:-- --:--:-- --:--:--  440k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  224k  100  224k    0     0   281k      0 --:--:-- --:--:-- --:--:--  281k


Next we can use a tool called gffread to create a transcriptome reference file using the gtf and genome files we downloaded.

In [44]:
!gffread -w data/reference/GCF_001632805.1_transcriptome_reference.fa -g data/reference/GCF_001632805.1_ASM163280v1_genomic.fna data/reference/GCF_001632805.1_ASM163280v1_genomic.gff

It is also recommended to include the full genome at the end of the transcriptome reference file, for the purpose of performing a 'decoy-aware' mapping, more information about which can be found in the Salmon documentation.

To alert the tool to the presence of this, we will also create a 'decoy file', which salmon needs pointed towards the full genome sequence in our transcriptome reference file.

In [45]:
!cat data/reference/GCF_001632805.1_transcriptome_reference.fa <(echo) data/reference/GCF_001632805.1_ASM163280v1_genomic.fna > data/reference/GCF_001632805.1_transcriptome_reference_w_decoy.fa
!echo "NZ_CP007220.1" > data/reference/decoys.txt

### STEP 5: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

The below code may take around 25 minutes to run.

In [69]:
#run fastqc for forward reads
!cat accs.txt | xargs -I {} fastqc "data/raw_fastq/{}_1.fastq" -o data/fastqc/
#run fastqc for reverse reads
!cat accs.txt | xargs -I {} fastqc "data/raw_fastq/{}_2.fastq" -o data/fastqc/

null
Started analysis of SRR13349122_1.fastq
Approx 5% complete for SRR13349122_1.fastq
Approx 10% complete for SRR13349122_1.fastq
Approx 15% complete for SRR13349122_1.fastq
Approx 20% complete for SRR13349122_1.fastq
Approx 25% complete for SRR13349122_1.fastq
Approx 30% complete for SRR13349122_1.fastq
Approx 35% complete for SRR13349122_1.fastq
Approx 40% complete for SRR13349122_1.fastq
Approx 45% complete for SRR13349122_1.fastq
Approx 50% complete for SRR13349122_1.fastq
Approx 55% complete for SRR13349122_1.fastq
Approx 60% complete for SRR13349122_1.fastq
Approx 65% complete for SRR13349122_1.fastq
Approx 70% complete for SRR13349122_1.fastq
Approx 75% complete for SRR13349122_1.fastq
Approx 80% complete for SRR13349122_1.fastq
Approx 85% complete for SRR13349122_1.fastq
Approx 90% complete for SRR13349122_1.fastq
Approx 95% complete for SRR13349122_1.fastq
Analysis complete for SRR13349122_1.fastq
null
Started analysis of SRR13349123_1.fastq
Approx 5% complete for SRR1334912

Fastqc will output the results in HTML format, as below, for all forward and reverse reads.


In [70]:
from IPython.display import IFrame
IFrame(src='./data/fastqc/SRR13349126_1_fastqc.html', width=800, height=600)

Although its best practice to look over them individually, tools like multiqc allow one to quickly look at a summary of the quality reports of the fastq files.

For instance, the below table shows which warnings, passes, or failures, from each fastqc report. There are other summaries created as well by multiqc. 

In [71]:
!multiqc -f data/fastqc/

import pandas as pd
dframe = pd.read_csv("./multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)


  /// ]8;id=797980;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.16

|           multiqc | Search path : /home/jupyter/data/fastqc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 48/48  c/SRR13349126_2_fastqc.html.html
|            fastqc | Found 24 reports
|           multiqc | Report      : multiqc_report.html   (overwritten)
|           multiqc | Data        : multiqc_data   (overwritten)
|           multiqc | MultiQC complete


,Sample,sequence_length_distribution,per_sequence_gc_content,Encoding,per_base_n_content,sequence_duplication_levels,Total Sequences,per_base_sequence_content,File type,Sequence length,...,basic_statistics,%GC,avg_sequence_length,per_base_sequence_quality,Filename,per_sequence_quality_scores,total_deduplicated_percentage,Total Bases,overrepresented_sequences,median_sequence_length
0,SRR13349122_1,pass,pass,Sanger / Illumina 1.9,pass,fail,10827590.0,fail,Conventional base calls,51.0,...,pass,55.0,51.0,pass,SRR13349122_1.fastq,pass,5.503069,552.2 Mbp,fail,51
1,SRR13349122_2,pass,pass,Sanger / Illumina 1.9,pass,fail,10827590.0,fail,Conventional base calls,51.0,...,pass,56.0,51.0,pass,SRR13349122_2.fastq,pass,9.335313,552.2 Mbp,fail,51
2,SRR13349123_1,pass,pass,Sanger / Illumina 1.9,pass,fail,11165256.0,fail,Conventional base calls,51.0,...,pass,56.0,51.0,pass,SRR13349123_1.fastq,pass,5.097632,569.4 Mbp,fail,51
3,SRR13349123_2,pass,warn,Sanger / Illumina 1.9,pass,fail,11165256.0,fail,Conventional base calls,51.0,...,pass,56.0,51.0,pass,SRR13349123_2.fastq,pass,9.070512,569.4 Mbp,fail,51
4,SRR13349124_1,pass,pass,Sanger / Illumina 1.9,pass,fail,10727273.0,fail,Conventional base calls,51.0,...,pass,55.0,51.0,pass,SRR13349124_1.fastq,pass,5.313842,547 Mbp,fail,51
5,SRR13349124_2,pass,pass,Sanger / Illumina 1.9,pass,fail,10727273.0,fail,Conventional base calls,51.0,...,pass,56.0,51.0,pass,SRR13349124_2.fastq,pass,9.139844,547 Mbp,fail,51
6,SRR13349125_1,pass,pass,Sanger / Illumina 1.9,pass,fail,10992686.0,fail,Conventional base calls,51.0,...,pass,55.0,51.0,pass,SRR13349125_1.fastq,pass,4.918485,560.6 Mbp,fail,51
7,SRR13349125_2,pass,warn,Sanger / Illumina 1.9,pass,fail,10992686.0,fail,Conventional base calls,51.0,...,pass,56.0,51.0,pass,SRR13349125_2.fastq,pass,8.541477,560.6 Mbp,fail,51
8,SRR13349126_1,pass,pass,Sanger / Illumina 1.9,pass,fail,12267497.0,fail,Conventional base calls,51.0,...,pass,55.0,51.0,pass,SRR13349126_1.fastq,pass,4.804107,625.6 Mbp,fail,51
9,SRR13349126_2,pass,warn,Sanger / Illumina 1.9,pass,fail,12267497.0,fail,Conventional base calls,51.0,...,pass,55.0,51.0,pass,SRR13349126_2.fastq,pass,9.482849,625.6 Mbp,fail,51


### STEP 5.1 Merging our fastq files

The following step may not be necessary -- it depends on the study.

In our case, if we look at our SRA files:

https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP300216&o=acc_s%3Aa

We will notice that, although there are 12 SRA files, coming from 12 SRR runs -- Actually, there are only 6 total samples. 

In such a case it is possible that, for instance, multiple different lanes in a flowcell may have been used for the same sample.

In our analysis will be comparing at the sample level. So, we would like to merge the fastq files that, although were created as separate fastq files by the sequencer, actually came from the sample. 

It is generally easier to do this merging after an initial fastqc report, as it makes it easier to pinpoint errors that may be lane specific.

Combining two FASTQ files is a straightforward process. Remember how FASTQ files are formatted, they are a list of readcounts. Consequently, we can simply 'concatenate' or add one fastq file to the bottom of another to create a merged fastq. Note that header information in a single fastq file may now contain different lane information -- however for our downstream processes this is acceptable. Remember if your fastq files are zipped, you will have to unzip them first.

In [24]:
#example of how to concatenate two of our fastq files from the same experiment.
!cat data/raw_fastq/SRR13349122_1.fastq data/raw_fastq/SRR13349123_1.fastq > data/raw_fastq/GSM5004088_1.fastq
!cat data/raw_fastq/SRR13349122_2.fastq data/raw_fastq/SRR13349123_2.fastq > data/raw_fastq/GSM5004088_2.fastq
#notice we concat the forward read with a forward read, and reverse with reverse.
#also note here we are naming it after the GSM, which is the sample experiment ID.



We could manually do the above for all 12 of our runs, and it wouldn't be much work.

If you are comfortable doing so, that is the best process to use.

As always though, there are ways to automate things. For instance, we could make use of our query code from 3.1.2 to obtain a list of the SRX IDs, and take advantage of our Jupyter's ability to write python to write a loop to iterate and concat our list.

Specific to our case, each sample contains two paired-end SRRs. 

Note; running this step will remove the previous unmerged fastq files in order to save space.

This will take about one hour.

In [25]:
###use query to get accession and sample ids; same code as in 3.1.2:
from google.cloud import bigquery
import pandas
client = bigquery.Client(location="US")
query = """
#standardSQL
SELECT *
FROM `nih-sra-datastore.sra.metadata`
WHERE organism = 'Mycobacteroides chelonae'
and acc LIKE '%SRR133491%'
"""
query_job = client.query(
    query,
    location="US",
)  
df = query_job.to_dataframe()
######################################################################

#import os so we can easily pass strings to shell commands using 'os.system'
import os

#now get the accession id's and sample id's from the created dataframe
runs = df['acc'].values
samples = list(set(df['sample_name'].values))

#sort them to be in numerical order
runs.sort()
samples.sort()

#now iterate through the samples, 
#because there are two SRRs to a run, 
#this means corresponding SRRs indices to an index of a GSM will be
#gsm index *2, and *2+1 
for index, item in enumerate(samples):
    
    #concatenate the two SRRs
    os.system(f"cat data/raw_fastq/{runs[index*2]}_1.fastq data/raw_fastq/{runs[index*2+1]}_1.fastq > data/raw_fastq/{samples[index]}_1.fastq")
    #delete the previous fastq files to save space
    os.system(f"rm data/raw_fastq/{runs[index*2]}_1.fastq")
    os.system(f"rm data/raw_fastq/{runs[index*2+1]}_1.fastq")
    #zip the merged fastq file to save more space
    os.system(f"gzip data/raw_fastq/{samples[index]}_1.fastq")
    
    #repeat for reverse reads
    os.system(f"cat data/raw_fastq/{runs[index*2]}_2.fastq data/raw_fastq/{runs[index*2+1]}_2.fastq > data/raw_fastq/{samples[index]}_2.fastq")
    
    os.system(f"rm data/raw_fastq/{runs[index*2]}_2.fastq")
    os.system(f"rm data/raw_fastq/{runs[index*2+1]}_2.fastq")  
   
    #its good practice to zip files to save space
    os.system(f"gzip data/raw_fastq/{samples[index]}_2.fastq")
   

In [26]:
#since our files will now be samples, not SRRs we can write a new text file to use for downstream batch processes.
#we can use the DF we made in the previous cell.
with open('samples.txt', 'w') as f:
    df = df.sort_values(by='sample_name', ascending=True)
    samples = df['sample_name'].unique()
    samples = '\n'.join(map(str, samples))
    f.write(samples)
    
!cat samples.txt

GSM5004088
GSM5004089
GSM5004090
GSM5004091
GSM5004092
GSM5004093

### STEP 5.3: Copy data file for Trimmomatic

One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [27]:
!gsutil -m cp -r gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa .
!head TruSeq3-PE.fa 

Copying gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa...
/ [1/1 files][   95.0 B/   95.0 B] 100% Done                                    
Operation completed over 1 objects/95.0 B.                                       
>PrefixPE/1
TACACTCTTTCCCTACACGACGCTCTTCCGATCT
>PrefixPE/2
GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT



### STEP 6: Run Trimmomatic
Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Using piping and our original list, it is possible to queue up a batch run of trimmomatic for all our files, note that this is a different way to run a loop compared with what we did before.

The below code may take approximately 35 minutes to run.

In [28]:
!cat samples.txt | xargs -I {} trimmomatic PE -threads $THREADS 'data/raw_fastq/{}_1.fastq.gz' 'data/raw_fastq/{}_2.fastq.gz' 'data/trimmed/{}_1_trimmed.fastq.gz' 'data/trimmed/{}_1_trimmed_unpaired.fastq.gz' 'data/trimmed/{}_2_trimmed.fastq.gz' 'data/trimmed/{}_2_trimmed_unpaired.fastq.gz' ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticPE: Started with arguments:
 -threads 3 data/raw_fastq/GSM5004088_1.fastq.gz data/raw_fastq/GSM5004088_2.fastq.gz data/trimmed/GSM5004088_1_trimmed.fastq.gz data/trimmed/GSM5004088_1_trimmed_unpaired.fastq.gz data/trimmed/GSM5004088_2_trimmed.fastq.gz data/trimmed/GSM5004088_2_trimmed_unpaired.fastq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 21992846 Both Surviving: 21906525 (99.61%) Forward Only Surviving: 86263 (0.39%) Reverse Only Surviving: 0 (0.00%) Dropped: 58 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with arguments:
 -threads 3 data/raw_fastq/GSM5004089_1.fastq.gz data/raw_fastq/GSM5004089_2.fastq.gz data/trimmed/GSM5004089_1_trimmed.fast

### STEP 7 (Optional): Run FastQC

It's best practice to run FastQC after trimming. However, you may decide to run FastQC only once, before or after trimming.

We will proceed with only the forward reads -- this is because, looking at trimmomatic, there were very few 'orphaned' reads. That is to say, most forward and reverse reads were successfully paired together. Because we are just trying to map to a transcriptome, the read lengths of the forward reads alone, in this case, around 50~ basepairs, should be sufficient.

The below code may take around 10 minutes to run.

In [29]:
!cat samples.txt | xargs -I {} fastqc "data/trimmed/{}_1_trimmed.fastq.gz" -o data/fastqc_samples/

application/gzip
Started analysis of GSM5004088_1_trimmed.fastq.gz
Approx 5% complete for GSM5004088_1_trimmed.fastq.gz
Approx 10% complete for GSM5004088_1_trimmed.fastq.gz
Approx 15% complete for GSM5004088_1_trimmed.fastq.gz
Approx 20% complete for GSM5004088_1_trimmed.fastq.gz
Approx 25% complete for GSM5004088_1_trimmed.fastq.gz
Approx 30% complete for GSM5004088_1_trimmed.fastq.gz
Approx 35% complete for GSM5004088_1_trimmed.fastq.gz
Approx 40% complete for GSM5004088_1_trimmed.fastq.gz
Approx 45% complete for GSM5004088_1_trimmed.fastq.gz
Approx 50% complete for GSM5004088_1_trimmed.fastq.gz
Approx 55% complete for GSM5004088_1_trimmed.fastq.gz
Approx 60% complete for GSM5004088_1_trimmed.fastq.gz
Approx 65% complete for GSM5004088_1_trimmed.fastq.gz
Approx 70% complete for GSM5004088_1_trimmed.fastq.gz
Approx 75% complete for GSM5004088_1_trimmed.fastq.gz
Approx 80% complete for GSM5004088_1_trimmed.fastq.gz
Approx 85% complete for GSM5004088_1_trimmed.fastq.gz
Approx 90% compl

### STEP 8: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

In [30]:
#!multiqc -f data/fastqc_samples/
!multiqc -f -o data/multiqc_samples/ data/fastqc_samples/


  /// ]8;id=652579;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.16

|           multiqc | Search path : /home/jupyter/data/fastqc_samples
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  0m  
|            fastqc | Found 6 reports
|           multiqc | Report      : data/multiqc_samples/multiqc_report.html
|           multiqc | Data        : data/multiqc_samples/multiqc_data
|           multiqc | MultiQC complete


### STEP 9: Index the Transcriptome so that Trimmed Reads Can Be Mapped Using Salmon

To run salmon we must specify the reference transcriptome, and the folder of our created index.

Note here, -i does not mean input, but the folder where the index will be created.

In [31]:
!salmon index -t data/reference/GCF_001632805.1_transcriptome_reference_w_decoy.fa -p $THREADS -i data/reference/transcriptome_index --decoys data/reference/decoys.txt -k 31 --keepDuplicates

Version Server Response: Not Found
[2023-10-07 00:26:49.279] [jLog] [info] building index
out : data/reference/transcriptome_index
[2023-10-07 00:26:49.280] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers

[2023-10-07 00:26:49.866] [puff::index::jointLog] [warning] There were 1 transcripts that would need to be removed to avoid duplicates.
[2023-10-07 00:26:49.888] [puff::index::jointLog] [info] Replaced 0 non-ATCG nucleotides
[2023-10-07 00:26:49.888] [puff::index::jointLog] [info] Clipped poly-A tails from 0 transcripts
wrote 4916 cleaned references
[2023-10-07 00:26:49.948] [puff::index::jointLog] [info] Filter size not provided; estimating from number of distinct k-mers
[2023-10-07 00:26:50.079] [puff::index::jointLog] [info] ntHll estimated 4966944 distinct k-mers, setting filter size to 2^27
Threads = 3
Vertex length = 31
Hash functions = 5
Filter size = 134217728
Capacity = 2
Files: 
data/reference/transcriptome_index/ref_k31_fixed.fa
-----------

### STEP 10: Run Salmon to Map Reads to Transcripts and Quantify Expression Levels
Salmon aligns the trimmed reads to the reference transcriptome and generates the read counts per transcript. In this analysis, each gene has a single transcript.

In [32]:
!cat samples.txt | xargs -I {} salmon quant -i data/reference/transcriptome_index -l SR -r "data/trimmed/{}_1_trimmed.fastq.gz" -p 8 --validateMappings -o "data/quants/{}_quant" > dump.txt

Version Server Response: Not Found
### salmon (selective-alignment-based) v1.10.2
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { data/reference/transcriptome_index }
### [ libType ] => { SR }
### [ unmatedReads ] => { data/trimmed/GSM5004088_1_trimmed.fastq.gz }
### [ threads ] => { 8 }
### [ validateMappings ] => { }
### [ output ] => { data/quants/GSM5004088_quant }
Logs will be written to data/quants/GSM5004088_quant/logs
[2023-10-07 00:26:58.893] [jointLog] [info] setting maxHashResizeThreads to 8
[2023-10-07 00:26:58.893] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2023-10-07 00:26:58.936] [jointLog] [info] Usage of --validateMappings implies use of minScoreFraction. Since not explicitly specified, it is being set to 0.65
[2023-10-07 00:26:58.936] [jointLog] [info] Setting consensusSlack to selective-alignment default of 0.35.
[2023-10-07 00:26:58.936] [jointLog] [info] parsing read library fo

### STEP 11: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [33]:
!head data/quants/SRR13349122_quant/quant.sf -n 1
!sort -nrk 4,4 data/quants/GSM5004088_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/GSM5004089_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/GSM5004090_quant/quant.sf | head -10


head: cannot open 'data/quants/SRR13349122_quant/quant.sf' for reading: No such file or directory
rna-BB28_RS07080	117	3.947	357338.048903	33036.000
rna-BB28_RS07075	3114	2864.000	229329.491561	15383189.000
rna-BB28_RS07070	1516	1266.000	131574.341979	3901381.000
rna-BB28_RS17330	369	119.000	74467.026985	207551.000
gene-BB28_RS02220	204	9.377	8550.974580	1878.000
gene-BB28_RS20695	231	14.032	7089.740556	2330.000
rna-BB28_RS09710	405	155.000	5854.026087	21252.000
gene-BB28_RS18745	300	51.326	3842.387771	4619.000
gene-BB28_RS18945	222	12.150	3654.546805	1040.000
gene-BB28_RS24750	102	3.543	3422.326286	284.000
sort: write failed: 'standard output': Broken pipe
sort: write error
rna-BB28_RS07080	117	3.947	462568.097698	48026.000
rna-BB28_RS07075	3114	2864.000	206010.239705	15519146.000
rna-BB28_RS07070	1516	1266.000	110132.380464	3667370.000
rna-BB28_RS17330	369	119.000	61183.074172	191507.000
gene-BB28_RS02220	204	9.377	5708.606906	1408.000
gene-BB28_RS20695	231	14.032	5570.638653	2056.00

Top 10 most highly expressed genes in the double lysogen samples.


In [34]:
!head data/quants/SRR13349122_quant/quant.sf -n 1
!sort -nrk 4,4 data/quants/GSM5004091_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/GSM5004092_quant/quant.sf | head -10
!sort -nrk 4,4 data/quants/GSM5004093_quant/quant.sf | head -10

head: cannot open 'data/quants/SRR13349122_quant/quant.sf' for reading: No such file or directory
rna-BB28_RS07080	117	3.947	445322.823351	49486.000
rna-BB28_RS07075	3114	2864.000	241952.879877	19508167.000
rna-BB28_RS07070	1516	1266.000	94029.240899	3351269.000
rna-BB28_RS17330	369	119.000	80359.275318	269213.000
gene-BB28_RS20695	231	14.032	5166.759244	2041.000
rna-BB28_RS09710	405	155.000	4668.399283	20371.000
gene-BB28_RS02220	204	9.377	3958.558949	1045.000
gene-BB28_RS14885	195	8.348	2910.564225	684.000
gene-BB28_RS06975	216	11.099	2672.393880	835.000
gene-BB28_RS21780	213	10.625	2330.298208	697.000
sort: write failed: 'standard output': Broken pipe
sort: write error
rna-BB28_RS07080	117	3.947	391434.811009	31215.000
rna-BB28_RS07075	3114	2864.000	277145.749054	16035791.000
rna-BB28_RS07070	1516	1266.000	96694.625060	2473120.000
rna-BB28_RS17330	369	119.000	78852.069744	189570.000
rna-BB28_RS09710	405	155.000	5005.402321	15674.000
gene-BB28_RS02220	204	9.377	4238.770223	803.000
ge

### STEP 12: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [35]:
!grep 'BB28_RS16545' data/quants/GSM5004088_quant/quant.sf
!grep 'BB28_RS16545' data/quants/GSM5004089_quant/quant.sf
!grep 'BB28_RS16545' data/quants/GSM5004090_quant/quant.sf

gene-BB28_RS16545	987	737.000	51.733340	893.000
gene-BB28_RS16545	987	737.000	43.538049	844.000
gene-BB28_RS16545	987	737.000	53.245339	939.000


Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [36]:
!grep 'BB28_RS16545' data/quants/GSM5004091_quant/quant.sf
!grep 'BB28_RS16545' data/quants/GSM5004092_quant/quant.sf
!grep 'BB28_RS16545' data/quants/GSM5004093_quant/quant.sf

gene-BB28_RS16545	987	737.000	5.590850	116.000
gene-BB28_RS16545	987	737.000	5.507281	82.000
gene-BB28_RS16545	987	737.000	5.590434	100.000


### STEP 12: Combine Genecounts to a Single Genecount File
Commonly, the readcounts for each sample are combined into a single table, where the rows contain the gene ID, and the columns identify the sample.

As before, this can be done in many ways. The quantmerge function outputs a table.

However, it is common for readcount tables to have sample headers for the columns, which this does not have.

So you could manually add those headers in different ways, for instance using a spreadsheet editor, or using a shell command like sed as below to insert a line at the start of table with the relevant tab-seperated headers.

You could also use sed to remove 'gene' or 'rna' prefixes from the table.

In [37]:
##first merge salmon files by number of reads.
!salmon quantmerge --column numreads --quants data/quants/*_quant -o data/quants/merged_quants.txt
##optinally we can rename the columns
!sed -i "1s/.*/Name\tGSM5004088\tGSM5004089\tGSM5004090\tGSM5004091\tGSM5004092\tGSM5004093/" data/quants/merged_quants.txt

##for further formatting, it may be easier in our r-code to later merge
##if we remove the gene- and rna- prefix
!sed -i "s/gene-//" data/quants/merged_quants.txt
!sed -i "s/rna-//" data/quants/merged_quants.txt

print("An example of a combined genecount outputfile.")
!head data/quants/merged_quants.txt

Version Server Response: Not Found
[2023-10-07 00:30:39.586] [mergeLog] [info] samples: [ data/quants/GSM5004088_quant, data/quants/GSM5004089_quant, data/quants/GSM5004090_quant, data/quants/GSM5004091_quant, data/quants/GSM5004092_quant, data/quants/GSM5004093_quant ]
[2023-10-07 00:30:39.586] [mergeLog] [info] sample names : [ GSM5004088_quant, GSM5004089_quant, GSM5004090_quant, GSM5004091_quant, GSM5004092_quant, GSM5004093_quant ]
[2023-10-07 00:30:39.586] [mergeLog] [info] output column : NUMREADS
[2023-10-07 00:30:39.586] [mergeLog] [info] output file : data/quants/merged_quants.txt
[2023-10-07 00:30:39.586] [mergeLog] [info] Parsing data/quants/GSM5004088_quant/quant.sf
[2023-10-07 00:30:39.595] [mergeLog] [info] Parsing data/quants/GSM5004089_quant/quant.sf
[2023-10-07 00:30:39.602] [mergeLog] [info] Parsing data/quants/GSM5004090_quant/quant.sf
[2023-10-07 00:30:39.609] [mergeLog] [info] Parsing data/quants/GSM5004091_quant/quant.sf
[2023-10-07 00:30:39.616] [mergeLog] [info

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://youtu.be/ChGfBR4do_Y>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)